# 03 - Selección de Escenario y Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import warnings

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

## 1. Selección de Escenario (Interactivo)

Por favor, selecciona el tipo de operación que deseas modelar. El resto del proyecto (entrenamiento y visualización) se basará en esta elección.

**(Importante: Después de elegir, continúa ejecutando las celdas de abajo).**

In [2]:
# Crea un dropdown para que el usuario elija
operation_widget = widgets.Dropdown(
    options=['Venta', 'Alquiler'],
    value='Venta', 
    description='Tipo de Operación:',
    disabled=False,
)
display(operation_widget)

Dropdown(description='Tipo de Operación:', options=('Venta', 'Alquiler'), value='Venta')

## 2. Carga y Filtrado de Datos

Cargamos los datos limpios y aplicamos el filtro seleccionado, incluyendo el filtro de moneda correspondiente.

In [3]:
# Cargar los datos limpios (generados por 01_data_cleaning)
try:
    df_clean = pd.read_csv('../data/processed/properati_clean.csv')
except FileNotFoundError:
    print("Error: No se encontró 'properati_clean.csv'. Asegúrate de ejecutar los notebooks anteriores.")
    df_clean = pd.DataFrame(columns=['operation_type', 'currency', 'price'])

# --- Capturar y Aplicar la Elección del Widget ---
OPERATION_CHOICE = operation_widget.value
print(f"Tipo de operación seleccionado: {OPERATION_CHOICE}")

# 1. Filtrar por tipo de operación
df_filtered = df_clean[df_clean['operation_type'] == OPERATION_CHOICE].copy()

# 2. Filtrar por moneda (CRÍTICO)
if OPERATION_CHOICE == 'Venta':
    CURRENCY_CHOICE = 'USD'
    TARGET_COLUMN = 'price_usd' # Nuevo nombre de columna
    df_filtered = df_filtered[df_filtered['currency'] == CURRENCY_CHOICE]
    # Renombrar 'price' a 'price_usd' para claridad
    df_filtered.rename(columns={'price': TARGET_COLUMN}, inplace=True)
else: # Asumir Alquiler
    CURRENCY_CHOICE = 'ARS'
    TARGET_COLUMN = 'price_ars' # Nuevo nombre de columna
    df_filtered = df_filtered[df_filtered['currency'] == CURRENCY_CHOICE]
    # Renombrar 'price' a 'price_ars'
    df_filtered.rename(columns={'price': TARGET_COLUMN}, inplace=True)

print(f"Filtrando por moneda: {CURRENCY_CHOICE}")
print(f"Datos originales (Venta y Alquiler): {len(df_clean)} filas")
print(f"Datos filtrados para '{OPERATION_CHOICE}': {len(df_filtered)} filas")

Tipo de operación seleccionado: Venta
Filtrando por moneda: USD
Datos originales (Venta y Alquiler): 5000 filas
Datos filtrados para 'Venta': 5000 filas


## 3. Ingeniería de Features (Feature Engineering)

Creamos nuevas variables (features) a partir de las existentes para mejorar el poder predictivo del modelo.

In [4]:
# 1. Ratios de Superficie
df_filtered['surface_ratio'] = df_filtered['surface_covered'] / df_filtered['surface_total']
# Manejar división por cero (si total es 0) y nulos
df_filtered['surface_ratio'] = df_filtered['surface_ratio'].replace([np.inf, -np.inf], 0).fillna(0)

# 2. Features de Texto (Simples)
df_filtered['es_nuevo'] = df_filtered['description'].str.contains('estrenar|nuevo', case=False, na=False).astype(int)
df_filtered['tiene_amenities'] = df_filtered['description'].str.contains('amenities|piscina|gimnasio|sum|pileta|cochera', case=False, na=False).astype(int)
df_filtered['es_lujo'] = df_filtered['description'].str.contains('lujo|premium|excelencia', case=False, na=False).astype(int)

# 3. Features de Interacción
df_filtered['total_rooms_bathrooms'] = df_filtered['rooms'].fillna(0) + df_filtered['bathrooms'].fillna(0)

print("Feature engineering completado. Nuevas columnas creadas:")
print(df_filtered[['surface_ratio', 'es_nuevo', 'tiene_amenities', 'es_lujo', 'total_rooms_bathrooms']].head())

Feature engineering completado. Nuevas columnas creadas:
   surface_ratio  es_nuevo  tiene_amenities  es_lujo  total_rooms_bathrooms
0       0.900000         0                0        0                    5.0
1       0.870130         0                1        0                    5.0
2       0.916667         0                0        0                    3.0
3       0.635135         0                0        0                    3.0
4       0.969697         0                1        0                    4.0


## 4. Guardar Dataset de Features

Seleccionamos solo las columnas que usará el modelo (features + target) y guardamos el archivo final.

In [5]:
# Lista de todas las columnas que queremos en el dataset final
features_to_keep = [
    'id', 'lat', 'lon', 'l1', 'l2', 'l3',
    'rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered',
    'property_type', 
    'surface_ratio', 'es_nuevo', 'tiene_amenities', 'es_lujo', 'total_rooms_bathrooms', # Nuevas features
    TARGET_COLUMN # La variable objetivo (price_usd o price_ars)
]

# Asegurarnos de que solo usamos columnas que realmente existen en el DataFrame
final_columns = [col for col in features_to_keep if col in df_filtered.columns]

df_final_features = df_filtered[final_columns]

# Importante: Eliminar filas donde el precio (target) es nulo
df_final_features = df_final_features.dropna(subset=[TARGET_COLUMN])

# Guardar el archivo que usará el notebook 04
df_final_features.to_csv('../data/processed/properati_features.csv', index=False)

print(f"Archivo 'properati_features.csv' guardado con {len(df_final_features)} registros.")
print(f"La columna objetivo (target) es: {TARGET_COLUMN}")

Archivo 'properati_features.csv' guardado con 5000 registros.
La columna objetivo (target) es: price_usd
